In [3]:
import numpy as np # linear algebra
import pandas as pd
import sqlite3

df_olist_customers = pd.read_csv('/content/olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('/content/olist_sellers_dataset.csv')
df_olist_order_reviews= pd.read_csv('/content/olist_order_reviews_dataset.csv')
df_olist_order_items= pd.read_csv('/content/olist_order_items_dataset.csv')
df_olist_products= pd.read_csv('/content/olist_products_dataset.csv')
df_olist_geolocation= pd.read_csv('/content/olist_geolocation_dataset.csv')
df_product_category_name_translation= pd.read_csv('/content/product_category_name_translation.csv')
df_olist_orders = pd.read_csv('/content/olist_orders_dataset.csv')
df_olist_order_payments= pd.read_csv('/content/olist_order_payments_dataset.csv')

df_olist_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_olist_customers.to_sql("olist_customers", con =engine)
df_olist_sellers.to_sql("olist_sellers", con =engine)
df_olist_order_reviews.to_sql("olist_order_reviews", con =engine)
df_olist_order_items.to_sql("olist_order_items", con =engine)
df_olist_products.to_sql("olist_products_dataset", con =engine)
df_olist_geolocation.to_sql("olist_geolocation", con =engine)
df_product_category_name_translation.to_sql("product_category_name_translation", con =engine)
df_olist_orders.to_sql("olist_orders", con =engine)
df_olist_order_payments.to_sql("olist_order_payments", con =engine)
df_olist_geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [5]:
sql='''

Select * from olist_customers
limit 5


''';

df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

,index,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [6]:
# create a function that will help me create queries more conveniently
def sqlq(query):
    return pd.read_sql_query(query, con=engine)

#     sql = 'SELECT * FROM olist_orders LIMIT 5'
# sqlq(sql)

In [7]:
sql = '''
SELECT
    COUNT(o.order_id) AS order_count,
    CAST(COUNT(o.order_id) AS REAL) * 100 / (SELECT COUNT(*) FROM olist_orders) AS percentage_of_total_orders
FROM olist_orders o
JOIN olist_order_reviews rev ON o.order_id = rev.order_id
WHERE strftime('%Y-%m', o.order_purchase_timestamp) = '2018-01'
AND rev.review_score = 5
'''
sqlq(sql)

,order_count,percentage_of_total_orders
0,4097,4.120031


In [8]:
sql = '''
SELECT
    strftime('%Y', o.order_purchase_timestamp) AS year,
    COUNT(DISTINCT o.customer_id) AS customer_count
FROM olist_orders o
GROUP BY year
ORDER BY year
'''
sqlq(sql)

,year,customer_count
0,2016,329
1,2017,45101
2,2018,54011


In [9]:
sql = '''
SELECT
    o.customer_id,
    AVG(p.payment_value) AS avg_order_value
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
GROUP BY o.customer_id
ORDER BY avg_order_value DESC
LIMIT 10
'''
sqlq(sql)

,customer_id,avg_order_value
0,1617b1357756262bfa56ab541c47bc16,13664.08
1,ec5b2ba62e574342386871631fafd3fc,7274.88
2,c6e2731c5b391845f6800c97401a43a9,6929.31
3,f48d464a0baaea338cb25f816991ab1f,6922.21
4,3fd6777bbce08a352fddd04e4a7cc8f6,6726.66
5,05455dfa7cd02f13d132aa7a6a9729c6,6081.54
6,df55c14d1476a9a3467f131269c2477f,4950.34
7,e0a2412720e9ea4f26c1ac985f6a7358,4809.44
8,24bbf5fd2f2e1b359ee7de94defc4a15,4764.34
9,3d979689f636322c62418b6346b1c6d2,4681.78


In [10]:
sql = '''
SELECT
    c.customer_city,
    SUM(p.payment_value) AS total_revenue
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY c.customer_city
ORDER BY total_revenue DESC
LIMIT 5
'''
sqlq(sql)

,customer_city,total_revenue
0,sao paulo,2203373.09
1,rio de janeiro,1161927.36
2,belo horizonte,421765.12
3,brasilia,354216.78
4,curitiba,247392.48


In [11]:
sql = '''
SELECT
    c.customer_state,
    SUM(p.payment_value) AS total_revenue
FROM olist_orders o
JOIN olist_order_payments p ON o.order_id = p.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY c.customer_state
ORDER BY total_revenue DESC

'''
sqlq(sql)

,customer_state,total_revenue
0,SP,5998226.96
1,RJ,2144379.69
2,MG,1872257.26
3,RS,890898.54
4,PR,811156.38
5,SC,623086.43
6,BA,616645.82
7,DF,355141.08
8,GO,350092.31
9,ES,325967.55


In [12]:
sql = '''
SELECT
    s.seller_id,
    SUM(p.payment_value) AS total_revenue,
    COUNT(DISTINCT o.customer_id) AS customer_count,
    COUNT(oi.order_id) AS goods_sold,
    SUM(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) AS five_star_reviews
FROM olist_sellers s
JOIN olist_order_items oi ON s.seller_id = oi.seller_id
JOIN olist_orders o ON oi.order_id = o.order_id
JOIN olist_order_payments p ON o.order_id = p.order_id
LEFT JOIN olist_order_reviews r ON o.order_id = r.order_id
GROUP BY s.seller_id
ORDER BY goods_sold DESC
LIMIT 10
'''
sqlq(sql)

,seller_id,total_revenue,customer_count,goods_sold,five_star_reviews
0,4a3ca9315b744ce9f8e9374361493884,306138.80,1806,2155,1021
1,6560211a19b47992c3666cc44a7e94c0,180028.11,1854,2130,1069
2,1f50f920176fa81dab994f9023523100,291918.98,1404,2017,1134
3,cc419e0650a3c5ba77189a1882b7556a,144535.03,1706,1893,1091
4,da8622b14eb17ae2831f4ac5b9dab84a,276578.63,1314,1662,948
5,955fee9216a65b617aa5c0531780ce60,236414.48,1287,1530,817
6,1025f0e2d44d7041d6cf58b6550e0bfa,312456.49,915,1477,750
7,7c67e1448b00f6e969d365cea6b010ab,512645.19,982,1463,489
8,7a67c85e85bb2ce8582c35f2203ad736,169734.22,1160,1245,752
9,ea8482cd71df3c1969d7b9473ff13abc,61708.19,1146,1240,638


In [13]:
sql = '''

SELECT
    c.customer_state,
    COUNT(o.order_id) AS total_orders,
    SUM(CASE WHEN o.order_status = 'delivered' AND o.order_delivered_customer_date IS NOT NULL THEN 1 ELSE 0 END) AS delivered_orders,
    ROUND(1.0 * SUM(CASE WHEN o.order_status = 'delivered' AND o.order_delivered_customer_date IS NOT NULL THEN 1 ELSE 0 END) / COUNT(o.order_id), 4) AS delivery_success_rate
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_state
ORDER BY delivery_success_rate DESC

'''
sqlq(sql)

,customer_state,total_orders,delivered_orders,delivery_success_rate
0,AC,81,80,0.9877
1,AP,68,67,0.9853
2,ES,2033,1995,0.9813
3,MS,715,701,0.9804
4,AM,148,145,0.9797
5,TO,280,274,0.9786
6,RS,5466,5344,0.9777
7,RN,485,474,0.9773
8,MT,907,886,0.9768
9,PR,5045,4923,0.9758


In [14]:
sql = '''

SELECT
    product_category_name_english,
    payment_type,
    payment_count
FROM (
    SELECT
        pct.product_category_name_english,
        p.payment_type,
        COUNT(*) AS payment_count,
        ROW_NUMBER() OVER (PARTITION BY pct.product_category_name_english ORDER BY COUNT(*) DESC) AS rn
    FROM olist_order_items oi
    JOIN olist_products_dataset prod ON oi.product_id = prod.product_id
    JOIN product_category_name_translation pct ON prod.product_category_name = pct.product_category_name
    JOIN olist_order_payments p ON oi.order_id = p.order_id
    GROUP BY pct.product_category_name_english, p.payment_type
) ranked
WHERE rn = 1
ORDER BY product_category_name_english;

'''
sqlq(sql)

,product_category_name_english,payment_type,payment_count
0,agro_industry_and_commerce,credit_card,145
1,air_conditioning,credit_card,222
2,art,credit_card,153
3,arts_and_craftmanship,credit_card,14
4,audio,credit_card,263
...,...,...,...
66,stationery,credit_card,1978
67,tablets_printing_image,credit_card,65
68,telephony,credit_card,3400
69,toys,credit_card,3294


In [15]:
sql = '''

WITH top_cities AS (
    SELECT
        c.customer_city,
        AVG(g.geolocation_lat) AS lat,
        AVG(g.geolocation_lng) AS lng
    FROM olist_customers c
    JOIN olist_geolocation g
      ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
    GROUP BY c.customer_city
    ORDER BY COUNT(*) DESC
    LIMIT 5
)
SELECT
    a.customer_city AS city_a,
    b.customer_city AS city_b,
    6371 * 2 *
        ASIN(
            SQRT(
                POWER(SIN(RADIANS(a.lat - b.lat) / 2), 2) +
                COS(RADIANS(a.lat)) * COS(RADIANS(b.lat)) *
                POWER(SIN(RADIANS(a.lng - b.lng) / 2), 2)
            )
        ) AS distance_km
FROM top_cities a
JOIN top_cities b
    ON a.customer_city < b.customer_city
ORDER BY distance_km;
'''
# df = pd.read_sql_query(sql, con=engine)'''
sqlq(sql)

,city_a,city_b,distance_km
0,niteroi,rio de janeiro,21.192146
1,santos,sao paulo,53.503635
2,rio de janeiro,santos,329.409203
3,belo horizonte,rio de janeiro,341.815464
4,belo horizonte,niteroi,342.258580
5,rio de janeiro,sao paulo,347.842527
6,niteroi,santos,350.361059
7,niteroi,sao paulo,369.034198
8,belo horizonte,sao paulo,491.675024
9,belo horizonte,santos,512.204480
